In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import sqlite3
import numpy as np
import re

In [ ]:
db = sqlite3.connect("nieruchomosciGratka.db")
cursor = db.cursor()
cursor.execute('INSERT TABLE offers (Cena REAL,Lokalicacja TEXT, Powierzchnia REAL,Pokoje TEXT,LPieter TEXT,Piętro TEXT,Rok TEXT, Piwnica TEXT, Balkon TEXT)')

In [ ]:
for i in range(1,100):
    print(f"Jestem na stronie: {i}")
    URL = "https://gratka.pl/nieruchomosci/mieszkania/krakow?page="+str(i)
    page = get(URL)
    bs=BeautifulSoup(page.content,'html.parser')
    for offers in bs.find_all('article',class_="teaserUnified",role="link"):
        link = offers["data-href"]
        pagelink = get(link)
        bslink=BeautifulSoup(pagelink.content,'html.parser')
        strona = bslink.find("ul",class_="parameters__singleParameters")
        cena = None
        r = bslink.find("div",class_='priceInfo')
        cena = r.find("span",class_="priceInfo__value").contents[0].replace(" ","")
        powierzchnia = None
        pietro = None
        pokoje = None
        liczbaPieter = None
        rok = None
        piwnica = 0
        balkon = 0
        lokalicacja = None
        stronaDodatkowa = bslink.find_all("li",class_="parameters__groupParameter")
        dodatkowa = list(map(lambda x:x.get_text().replace(" ","").replace("\n",""),stronaDodatkowa))
        if 'balkon' in dodatkowa:
            balkon += 1            
        if 'piwnica' in dodatkowa:
            piwnica += 1
        for dana in strona.find_all("li"):
            if dana.find("span")==None:
                continue
            else:
                if dana.find("span").get_text()=='Lokalizacja':
                    lokalicacja = dana.find(class_="parameters__value").get_text().replace(" ","").split(',')[1]
                if dana.find("span").get_text()=="Powierzchnia w m2":
                    powierzchnia = float(dana.find(class_="parameters__value").get_text().replace("m2","").replace(",","."))
                elif dana.find("span").get_text()=="Piętro":
                    pietro = dana.find(class_="parameters__value").get_text()
                elif dana.find("span").get_text()=="Liczba pokoi":
                    pokoje = (dana.find(class_="parameters__value").get_text())
                elif dana.find("span").get_text()=="Liczba pięter w budynku":
                    liczbaPieter = (dana.find(class_="parameters__value").get_text())
                elif dana.find("span").get_text()=="Rok budowy":
                    rok = dana.find(class_="parameters__value").get_text()
        cursor.execute('INSERT INTO  offers VALUES (?,?,?,?,?,?,?,?,?)',(cena,lokalicacja,powierzchnia,pokoje,liczbaPieter,pietro,rok,piwnica,balkon))
        db.commit()
db.close()